In [5]:
import simpy
import numpy as np

In [6]:
class JusticeSystem():
    def __init__(self, env) -> None:
        self.env = env

        # The numbers to track
        self.active_cases = 0
        self.magistrate_cases = 0
        self.crown_cases = 0

        # Some random, hard-coded constants
        self.summoned_dismissed_prob = 0.3
        self.magistrate_dismissed_prob = 0.3
        self.magistrate_sentenced_prob = 0.5 # This is after not dismissing. I.e., 0.5 of the `legitimate` cases got sentenced


    # The justice process
    def justice(self, env):
        while True:
            # Influx of cases
            self.active_cases += 1

            # State at the beginning of the day
            print(f'At the beginning of day {env.now+1}, the number of active cases is {self.active_cases},000 cases, cases in magistrate backlog is {self.magistrate_cases},000 cases, cases in crown court backlog is {self.crown_cases},000 cases.')

            yield self.env.process(self.summoned())

            if self.magistrate_cases > 0:
                yield self.env.process(self.magistrate()) 
            
            yield self.env.timeout(1)

    # The summoned process
    def summoned(self):
        # Cases dismissed
        if np.random.uniform() < self.summoned_dismissed_prob:
            self.active_cases -= 1
            yield self.env.timeout(0)
        
        # Cases moved to magistrate
        else:
            self.magistrate_cases += 1
            yield self.env.timeout(0) 

    # The magistrate process
    def magistrate(self):
        # Cases dismissed
        if np.random.uniform() < self.magistrate_dismissed_prob:            
            self.magistrate_cases -= 1
            self.active_cases -= 1
            yield self.env.timeout(0)    

        else:
            # Cases sentenced
            if np.random.uniform() < self.magistrate_sentenced_prob:
                self.magistrate_cases -= 1
                self.active_cases -= 1
                yield self.env.timeout(0)

            # Cases moved to crown court
            else:
                self.magistrate_cases -= 1
                self.crown_cases += 1
                yield self.env.timeout(0)


    # the crown process
    

In [7]:
def simulate(env):
    justice_system = JusticeSystem(env)
    env.process(justice_system.justice(env)) #Add process `summoned` to the simpy env
    
    yield env.timeout(0)

In [8]:
env = simpy.Environment()

env.process(simulate(env)) # Add process `simulate` to the simpy env
env.run(until=100)

At the beginning of day 1, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 2, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 3, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 4, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 5, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 6, the number of active cases is 1,000 cases, cases in magistrate backlog is 0,000 cases, cases in crown court backlog is 0,000 cases.
At the beginning of day 7, the number of activ